## AI Project
# Hammad Qaiser     Roll No: 21I-2463


In [7]:
import random
from tabulate import tabulate

MAX_COURSES = 10
MAX_SECTIONS = 4
MAX_PROFESSORS = 8
MAX_ROOMS = 4
MAX_DAYS = 5
MAX_TIMESLOTS = 16  # 8 timeslots from 8:30am to 5:30pm

class Course:
    def __init__(self, name, is_lab, instructors, max_students):
        self.name = name
        self.is_lab = is_lab
        self.instructors = instructors
        self.max_students = max_students

class Section:
    def __init__(self, name, strength):
        self.name = name
        self.strength = strength 

class Professor:
    def __init__(self, name, max_courses):
        self.name = name
        self.max_courses = max_courses

class Room:
    def __init__(self, name, capacity, room_type):
        self.name = name
        self.capacity = capacity
        self.room_type = room_type

class TimeSlot:
    def __init__(self, day, start_time, end_time):
        self.day = day
        self.start_time = start_time
        self.end_time = end_time

def generate_random_timetable(courses, sections, professors, rooms):
    timetable = []
    for _ in range(len(courses)):
        course_name = random.choice(courses).name
        section_name = random.choice(sections).name
        professor_name = random.choice(professors).name
        room_name = random.choice(rooms).name
        for day in range(1, MAX_DAYS + 1):  # Loop over all working days
            timeslot_start = random.randint(1, MAX_TIMESLOTS - 3)  # Ensure there's enough time for the lecture/lab
            timeslot_end = timeslot_start + (3 if "Lab" in course_name else 1)  # 3 timeslots for labs, 1 for lectures
            timeslot_str = f"{timeslot_start * 80 // 60}:00-{timeslot_end * 80 // 60}:20"  # Convert timeslots to time
            timetable_entry = (course_name, section_name, professor_name, room_name, day, timeslot_str)
            timetable.append(timetable_entry)

    return timetable


def calculate_fitness(timetable):
    hard_constraint_conflicts = 0
    soft_constraint_conflicts = 0

    professor_assignments = {}
    section_assignments = {}
    room_assignments = {}

    for entry in timetable:
        course_name, section_name, professor_name, room_name, day, timeslot = entry

        # Check for conflicts with professor assignments
        if professor_name in professor_assignments.get((day, timeslot), []):
            hard_constraint_conflicts += 1
        else:
            professor_assignments.setdefault((day, timeslot), []).append(professor_name)

        # Check for conflicts with section assignments
        if section_name in section_assignments.get((day, timeslot), []):
            hard_constraint_conflicts += 1
        else:
            section_assignments.setdefault((day, timeslot), []).append(section_name)

        # Check for conflicts with room assignments
        if room_name in room_assignments.get((day, timeslot), []):
            hard_constraint_conflicts += 1
        else:
            room_assignments.setdefault((day, timeslot), []).append(room_name)

    # Soft constraint: A class should be held in the same classroom across the whole week
    room_usage = {}  # Track room usage across the week
    for entry in timetable:
        _, _, _, room_name, day, _ = entry
        if room_name in room_usage:
            room_usage[room_name].add(day)
        else:
            room_usage[room_name] = {day}
    for room, days in room_usage.items():
        if len(days) != MAX_DAYS:
            soft_constraint_conflicts += 1

    for entry in timetable:
        _, _, _, _, _, timeslot = entry
        if "Lab" in entry[0] and not timeslot.endswith("pm"):
            soft_constraint_conflicts += 1
        elif "Lab" not in entry[0] and timeslot.endswith("pm"):
            soft_constraint_conflicts += 1

    # Calculate fitness based on hard constraints
    fitness = -hard_constraint_conflicts

    # Penalize fitness for each soft constraint conflict
    fitness -= soft_constraint_conflicts

    # Return fitness as a float
    return float(fitness)



def tournament_selection(population, tournament_size):
    selected_parents = []
    for _ in range(len(population)):
        tournament = random.sample(population, tournament_size)
        winner = max(tournament, key=lambda x: calculate_fitness(x))
        selected_parents.append(winner)
    return selected_parents

def one_point_crossover(parent1, parent2):
    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

def mutation(timetable, mutation_rate):
    mutated_timetable = timetable[:]
    for i in range(len(mutated_timetable)):
        if random.random() < mutation_rate:
            course_name, section_name, professor_name, room_name, day, timeslot = mutated_timetable[i]
            mutated_timetable[i] = (
                course_name,
                random.choice(sections).name,
                random.choice(professors).name,
                random.choice(rooms).name,
                random.randint(1, MAX_DAYS),
                f"{random.randint(1, MAX_TIMESLOTS - 3) * 80 // 60}:00-{random.randint(1, MAX_TIMESLOTS) * 80 // 60}:20"
            )
    return mutated_timetable

if __name__ == "__main__":
    courses = [
        Course(f"C{i}", random.choice([True, False]), ["Professor A", "Professor B"], random.randint(40, 100))
        for i in range(1, MAX_COURSES + 1)
    ]

    sections = [
        Section(chr(65 + i), random.randint(30, 60))
        for i in range(MAX_SECTIONS)
    ]

    professors = [
        Professor(f"Professor {chr(65 + i)}", random.randint(2, 4))
        for i in range(MAX_PROFESSORS)
    ]

    rooms = [
        Room(f"Room {i}", random.randint(30, 100), "Lecture" if i % 2 == 0 else "Lab")
        for i in range(1, MAX_ROOMS + 1)
    ]

    population_size = 20
    tournament_size = 5
    mutation_rate = 0.1

    # Generate initial population
    population = [generate_random_timetable(courses, sections, professors, rooms) for _ in range(population_size)]

    # Evolution loop
    for generation in range(100):
        # Tournament selection
        selected_parents = tournament_selection(population, tournament_size)

        # Crossover
        children = []
        for i in range(0, len(selected_parents), 2):
            parent1, parent2 = selected_parents[i], selected_parents[i + 1]
            child1, child2 = one_point_crossover(parent1, parent2)
            children.extend([child1, child2])

        # Mutation
        mutated_children = [mutation(child, mutation_rate) for child in children]

        # Replace population with children
        population = mutated_children

        # Display generation's best fitness and first chromosome
        best_timetable = max(population, key=lambda x: calculate_fitness(x))
        fitness = calculate_fitness(best_timetable)
        print(f"Generation {generation + 1} - Best Fitness: {fitness}")
        print("Best Timetable:")
        headers = ["Course Name", "Section", "Professor", "Room", "Day", "Time Slot"]
        data = []
        for entry in best_timetable:
            data.append([entry[0], entry[1], entry[2], entry[3], f"Day {entry[4]}", entry[5]])
        print(tabulate(data, headers=headers, tablefmt="grid"))
        print()

Generation 1 - Best Fitness: -2.0
Best Timetable:
+---------------+-----------+-------------+--------+-------+-------------+
| Course Name   | Section   | Professor   | Room   | Day   | Time Slot   |
+===============+===========+=============+========+=======+=============+
| C1            | B         | Professor D | Room 1 | Day 1 | 2:00-4:20   |
+---------------+-----------+-------------+--------+-------+-------------+
| C1            | A         | Professor D | Room 4 | Day 2 | 5:00-1:20   |
+---------------+-----------+-------------+--------+-------+-------------+
| C1            | B         | Professor D | Room 1 | Day 3 | 8:00-9:20   |
+---------------+-----------+-------------+--------+-------+-------------+
| C1            | B         | Professor D | Room 1 | Day 4 | 14:00-16:20 |
+---------------+-----------+-------------+--------+-------+-------------+
| C1            | B         | Professor D | Room 1 | Day 5 | 13:00-14:20 |
+---------------+-----------+-------------+-------